# Batch Normalization (from scratch)
This notebook walks through *what BatchNorm does* and how the provided `Layer_BatchNormalization` works.

We focus on the most common 2D case: inputs shaped `(batch, features)` (e.g., activations of an MLP layer).


## 1) Why normalization helps
BatchNorm stabilizes training by normalizing activations using **batch statistics** (mean/variance), then learning an affine re-scale (`gamma`) and shift (`beta`).

Training-time (per feature):
- `mu = mean(x)` over the batch
- `var = var(x)` over the batch
- `x_hat = (x - mu) / sqrt(var + eps)`
- `y = gamma * x_hat + beta`

Inference-time: use **running mean/var** accumulated during training.


In [2]:
import numpy as np
from batchnorm import Layer_BatchNormalization

np.random.seed(0)
x = np.random.randn(64, 10) * 5 + 3  # deliberately non-zero mean/variance
bn = Layer_BatchNormalization(n_features=10)

y = bn.forward(x, training=True)
print('training output mean (per feature):', y.mean(axis=0))
print('training output var  (per feature):', y.var(axis=0))

training output mean (per feature): [-8.50014503e-17  5.02527707e-17 -1.82145965e-16 -2.42861287e-17
  1.67400815e-16 -2.94035629e-16  8.67361738e-18 -1.63064007e-16
  1.35308431e-16  5.55111512e-17]
training output var  (per feature): [0.99999963 0.99999956 0.99999955 0.99999961 0.99999962 0.99999954
 0.99999954 0.99999965 0.99999962 0.99999951]


## 2) Running statistics
During training, we maintain exponentially-decayed running estimates.
These are used when `training=False`.


In [3]:
# Run multiple batches to update running stats
for _ in range(20):
    xb = np.random.randn(64, 10) * 2 + 1
    _ = bn.forward(xb, training=True)

print('running_mean:', bn.running_mean)
print('running_var :', bn.running_var)

# Inference: normalization uses running stats
x_test = np.random.randn(8, 10) * 2 + 1
y_test = bn.forward(x_test, training=False)
print('inference output shape:', y_test.shape)

running_mean: [[0.86773458 0.8619657  0.83573998 0.79636782 0.86087148 0.86764978
  0.9859476  0.91650534 0.92824561 0.89680214]]
running_var : [[3.90176783 3.98873478 3.92112058 3.78750281 3.84205094 3.80501912
  3.70500786 3.88168344 4.00293304 3.61602823]]
inference output shape: (8, 10)


## 3) Backprop intuition
BatchNorm’s backward pass has three learnable gradients:
- `dgamma` and `dbeta` are straightforward
- `dinputs` is trickier because `mu` and `var` depend on all samples in the batch

The implementation in `batchnorm.py` uses the standard analytic derivative.


## 4) Quick check: run the gradient test
The repo includes a finite-difference check to validate the backward pass.


In [4]:
# In terminal, run:
# python -m BatchNorm.test_batchnorm
# (This notebook doesn't auto-run it.)